In [1]:
from mlp import data_loader, metrics,wrapper,transformer,model,preprocessor,rounder

In [3]:

# read data
train, test, train_labels, specs, sample_submission = data_loader.read_data()
# get usefull dict with maping encode
train, test, train_labels, win_code, list_of_user_activities, list_of_event_code, activities_labels, assess_titles, list_of_event_id, list_title_event_code = data_loader.encode_title(train, test, train_labels)
# tranform function to get the train and test set
reduce_train, reduce_test, categoricals = data_loader.get_train_and_test(train,test, assess_titles,list_of_event_code,list_of_event_id,activities_labels,list_title_event_code,win_code)

Reading train.csv file....
Training.csv file have 11341042 rows and 11 columns
Reading test.csv file....
Test.csv file have 1156414 rows and 11 columns
Reading train_labels.csv file....
Train_labels.csv file have 17690 rows and 7 columns
Reading specs.csv file....
Specs.csv file have 386 rows and 3 columns
Reading sample_submission.csv file....
Sample_submission.csv file have 1000 rows and 2 columns
  5%|▌         | 882/17000 [01:03<11:12, 23.98it/s]

KeyboardInterrupt: 

In [0]:

# call feature engineering function
reduce_train, reduce_test, features = preprocessor.preprocess(reduce_train, reduce_test)

In [0]:
params = {'n_estimators':2000,
            'boosting_type': 'gbdt',
            'objective': 'regression',
            'metric': 'rmse',
            'subsample': 0.75,
            'subsample_freq': 1,
            'learning_rate': 0.04,
            'feature_fraction': 0.9,
         'max_depth': 15,
            'lambda_l1': 1,  
            'lambda_l2': 1,
            'verbose': 100,
            'early_stopping_rounds': 100, 'eval_metric': 'cappa'
            }

In [0]:
y = reduce_train['accuracy_group']

In [0]:
n_fold = 5
folds = GroupKFold(n_splits=n_fold)

In [0]:
cols_to_drop = ['game_session', 'installation_id', 'timestamp', 'accuracy_group', 'timestampDate']

In [0]:
mt = transformer.MainTransformer()
ft = transformer.FeatureTransformer()
transformers = {'ft': ft}
regressor_model1 = model.RegressorModel(model_wrapper=LGBWrapper_regr())
regressor_model1.fit(X=reduce_train, y=y, folds=folds, params=params, preprocesser=mt, transformers=transformers,
                    eval_metric='cappa', cols_to_drop=cols_to_drop)

In [0]:
%%time
pr1 = regressor_model1.predict(reduce_train)

optR = rounder.OptimizedRounder()
optR.fit(pr1.reshape(-1,), y)
coefficients = optR.coefficients()

In [0]:
opt_preds = optR.predict(pr1.reshape(-1, ), coefficients)
qwk(y, opt_preds)

In [0]:
# some coefficients calculated by me.
pr1 = regressor_model1.predict(reduce_test)
pr1[pr1 <= 1.12232214] = 0
pr1[np.where(np.logical_and(pr1 > 1.12232214, pr1 <= 1.73925866))] = 1
pr1[np.where(np.logical_and(pr1 > 1.73925866, pr1 <= 2.22506454))] = 2
pr1[pr1 > 2.22506454] = 3

In [0]:
sample_submission['accuracy_group'] = pr1.astype(int)
sample_submission.to_csv('submission.csv', index=False)

In [0]:
sample_submission['accuracy_group'].value_counts(normalize=True)